# Multiple inheritance

Python supports multiple inheritance, i.e., a class can have multiple parent classes.  Here, we will explore how Python deals with multiple inheritance.

## Method resoution order (MRO)

Consider multiple inheritance where a "diamond" occurs in the dependency diagram.

In [6]:
class BaseClass:
    
    def __repr__(self):
        return 'BaseClass'

Both `FirstLevelClass1` and `FirstLevelClass2` are derived from `BaseClass` and override the `__repr__` method.

In [7]:
class FirstLevelClass1(BaseClass):
    
    def __repr__(self):
        return 'Level 1.1'

In [8]:
class FirstLevelClass2(BaseClass):
    
    def __repr__(self):
        return 'Level 1.2'

In order to see the method resolution order in action, two classes are derived from both `FirstLevelClass1` and `FirstLevelClass2`.  `MyClass2` has `FirstLevelClass1` as the first parent

In [22]:
class MyClass1(FirstLevelClass1, FirstLevelClass2):
    pass

In [23]:
print(MyClass1())

Level 1.1


Class `MyClass2` on the other hand has `FirstLevelClass2` as its first parent.

In [13]:
class MyClass2(FirstLevelClass2, FirstLevelClass1):
    pass

In [14]:
print(MyClass2())

Level 1.2


In fact, every Python class has the `mro` method that will return a list of classes that will be used for method resolution in the order they appear in the list.  For `MyClass`, the first parent class to check will be `FirstLevelClass1`, while for `MyClass2` it is `FirstLevelClass2`, as we have already observed by calling the `__repr__` method.

In [17]:
MyClass1.mro()

[__main__.MyClass1,
 __main__.FirstLevelClass1,
 __main__.FirstLevelClass2,
 __main__.BaseClass,
 object]

In [19]:
MyClass2.mro()

[__main__.MyClass2,
 __main__.FirstLevelClass2,
 __main__.FirstLevelClass1,
 __main__.BaseClass,
 object]

## Using `super`

The `super` function can be used to call methods implemented by an ancestor class.  For instance, in class `MyClass1` we now override the `__repr__` method.

In [36]:
class MyClass1(FirstLevelClass1, FirstLevelClass2):
    
    def __repr__(self):
        return 'MyClass1: ' + super().__repr__()

In [37]:
print(MyClass1())

MyClass1: Level 1.1


The semantics is that the classes are searched in the MRO order, starting from the first parent class.  However, it is possible to control this by providing additional arguments to the `super` function.  For instance, if we wanted to use the `__repr__` implementation of `FirstLevelClass2` rather than that of `FirstLevelClass1`, that can be done as follows.

In [40]:
class MyClass12(FirstLevelClass1, FirstLevelClass2):
    
    def __repr__(self):
        return 'MyClass1: ' + super(FirstLevelClass1, self).__repr__()

In [11]:
class MyClass1(FirstLevelClass1, FirstLevelClass2):
    pass

In [12]:
print(MyClass1())

Level 1.1


In [41]:
print(MyClass12())

MyClass1: Level 1.2


This may seem counter-intuitive at first, but that is due to the semantics of the `super` function.  The method search will start after the class specified (in MRO order), so the first class to search for a `__repr__` implementation would be `FirstLevelClass2`, since that is the one after `FirstLevelClass1`, the first argument of `super`.

Similarly, to get the `__repr__` implementation of `BaseClass`, we would have to pass `FirstLevelClass2` to `super` so that the search starts in `BaseClass`.

In [42]:
class MyClassBase(FirstLevelClass1, FirstLevelClass2):
    
    def __repr__(self):
        return 'MyClass1: ' + super(FirstLevelClass2, self).__repr__()

In [43]:
print(MyClassBase())

MyClass1: BaseClass


Note that this is quite error prone.  If one of the classes in the hierarchy is redefined, this type of code is likely to break.

## Querying types

The `type` function will return the class an object is an instance of, e.g.,

In [44]:
obj = MyClass1()
type(obj)

__main__.MyClass1

Each object (at least in Python 3) also has a `__class__` attribute that hold the same information, e.g.,

In [45]:
obj.__class__

__main__.MyClass1

Hence it is easy to retrieve all ancestor classes of an object.

In [46]:
obj.__class__.mro()

[__main__.MyClass1,
 __main__.FirstLevelClass1,
 __main__.FirstLevelClass2,
 __main__.BaseClass,
 object]

However, in general you typically want to check whether an object is an instance of some class, and Python provides the `isinstance` function for that purpose.

In [48]:
isinstance(obj, FirstLevelClass1)

True

In [49]:
isinstance(obj, MyClassBase)

False

Obviously, an object is an instance of each class in its MRO, i.e.,

In [53]:
for cls in obj.__class__.mro():
    if isinstance(obj, cls):
        print(f'my object is instance of {cls.__name__}')
    else:
        print(f'weird for {cls.__name__}')

my object is instance of MyClass1
my object is instance of FirstLevelClass1
my object is instance of FirstLevelClass2
my object is instance of BaseClass
my object is instance of object


At the class level, the function `issubclass` serves a similar purpose.

In [50]:
issubclass(MyClass2, BaseClass)

True

In [51]:
issubclass(FirstLevelClass1, MyClass12)

False

In [54]:
for cls in MyClass1.mro():
    if issubclass(MyClass1, cls):
        print(f'my class is sbuclass of {cls.__name__}')
    else:
        print(f'weird for {cls.__name__}')

my class is sbuclass of MyClass1
my class is sbuclass of FirstLevelClass1
my class is sbuclass of FirstLevelClass2
my class is sbuclass of BaseClass
my class is sbuclass of object


Note that a class is a subclass of itself.